In [1]:
import os
os.chdir("../")

In [2]:
from project import logger
from project.utils.common import read_yaml_file,clean_data

import mlflow,boto3,pandas as pd,joblib

In [3]:
config=read_yaml_file("config/config.yaml")
model_name=config.mlflow.model_name
tracking_uri=config.mlflow.tracking_uri

In [4]:
mlflow.set_tracking_uri(tracking_uri)

In [5]:
text='i am feeling realy good today'

In [6]:
df=pd.DataFrame({'text':pd.Series(text)})
df=clean_data(df,'text')
clened_text=df['text']

In [7]:
def load_latest_model_and_artifacts():
    client = mlflow.tracking.MlflowClient()
    
    try:
        # Get the latest version of the model
        latest_versions = client.get_latest_versions(model_name, stages=["None"])
        
        if not latest_versions:
            raise ValueError(f"No registered versions found for model {model_name}.")
        
        latest_version = latest_versions[-1]
        run_id = latest_version.run_id

        # Load the latest model
        model_uri = f"runs:/{run_id}/model"
        model = mlflow.pyfunc.load_model(model_uri)
        
        local_artifact_path = os.path.join('artifacts', run_id)
        os.makedirs(local_artifact_path, exist_ok=True)
        client.download_artifacts(run_id, "vectorizer.pkl",local_artifact_path)
        
        logger.info(f"Loaded model from {model_uri}")
        logger.info(f"Downloaded artifacts to {local_artifact_path}")
        
        return model, local_artifact_path
    
    except mlflow.exceptions.RestException as e:
        logger.error(f"Error during loading model and artifacts: {e}")
        raise e
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        raise e

# Example usage
# model, artifacts_path = load_latest_model_and_artifacts("sentiment_classifier", artifact_subpath="specific/path")
# model, artifacts_path = load_latest_model_and_artifacts("sentiment_classifier")


In [8]:
model,artfiact_path=load_latest_model_and_artifacts()

In [9]:
vectorizer=joblib.load(os.path.join(artfiact_path,"vectorizer.pkl"))

In [11]:
clened_text=vectorizer.transform(df['text'])
clened_text=clened_text.toarray()

In [12]:
model.predict(clened_text)

array(['positive'], dtype=object)